In [24]:
from PIL import Image, ImageColor
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.spatial import KDTree
import os
import warnings

# Filter out the specific FutureWarning from seaborn
warnings.filterwarnings('ignore', category=FutureWarning, message='Passing `palette` without assigning `hue`.*')

# Your color and KDTree initialization remains the same.
colors = ['#000000', '#000033', '#000066', '#000099', '#0000CC', '#0000FF',
          '#003300', '#003333', '#003366', '#003399', '#0033CC', '#0033FF',
          '#006600', '#006633', '#006666', '#006699', '#0066CC', '#0066FF',
          '#009900', '#009933', '#009966', '#009999', '#0099CC', '#0099FF',
          '#00CC00', '#00CC33', '#00CC66', '#00CC99', '#00CCCC', '#00CCFF',
          '#00FF00', '#00FF33', '#00FF66', '#00FF99', '#00FFCC', '#00FFFF',
          '#330000', '#330033', '#330066', '#330099', '#3300CC', '#3300FF',
          '#333300', '#333333', '#333366', '#333399', '#3333CC', '#3333FF',
          '#336600', '#336633', '#336666', '#336699', '#3366CC', '#3366FF',
          '#339900', '#339933', '#339966', '#339999', '#3399CC', '#3399FF',
          '#33CC00', '#33CC33', '#33CC66', '#33CC99', '#33CCCC', '#33CCFF',
          '#33FF00', '#33FF33', '#33FF66', '#33FF99', '#33FFCC', '#33FFFF',
          '#660000', '#660033', '#660066', '#660099', '#6600CC', '#6600FF',
          '#663300', '#663333', '#663366', '#663399', '#6633CC', '#6633FF',
          '#666600', '#666633', '#666666', '#666699', '#6666CC', '#6666FF',
          '#669900', '#669933', '#669966', '#669999', '#6699CC', '#6699FF',
          '#66CC00', '#66CC33', '#66CC66', '#66CC99', '#66CCCC', '#66CCFF',
          '#66FF00', '#66FF33', '#66FF66', '#66FF99', '#66FFCC', '#66FFFF',
          '#990000', '#990033', '#990066', '#990099', '#9900CC', '#9900FF',
          '#993300', '#993333', '#993366', '#993399', '#9933CC', '#9933FF',
          '#996600', '#996633', '#996666', '#996699', '#9966CC', '#9966FF',
          '#999900', '#999933', '#999966', '#999999', '#9999CC', '#9999FF',
          '#99CC00', '#99CC33', '#99CC66', '#99CC99', '#99CCCC', '#99CCFF',
          '#99FF00', '#99FF33', '#99FF66', '#99FF99', '#99FFCC', '#99FFFF',
          '#CC0000', '#CC0033', '#CC0066', '#CC0099', '#CC00CC', '#CC00FF',
          '#CC3300', '#CC3333', '#CC3366', '#CC3399', '#CC33CC', '#CC33FF',
          '#CC6600', '#CC6633', '#CC6666', '#CC6699', '#CC66CC', '#CC66FF',
          '#CC9900', '#CC9933', '#CC9966', '#CC9999', '#CC99CC', '#CC99FF',
          '#CCCC00', '#CCCC33', '#CCCC66', '#CCCC99', '#CCCCCC', '#CCCCFF',
          '#CCFF00', '#CCFF33', '#CCFF66', '#CCFF99', '#CCFFCC', '#CCFFFF',
          '#FF0000', '#FF0033', '#FF0066', '#FF0099', '#FF00CC', '#FF00FF',
          '#FF3300', '#FF3333', '#FF3366', '#FF3399', '#FF33CC', '#FF33FF',
          '#FF6600', '#FF6633', '#FF6666', '#FF6699', '#FF66CC', '#FF66FF',
          '#FF9900', '#FF9933', '#FF9966', '#FF9999', '#FF99CC', '#FF99FF',
          '#FFCC00', '#FFCC33', '#FFCC66', '#FFCC99', '#FFCCCC', '#FFCCFF',
          '#FFFF00', '#FFFF33', '#FFFF66', '#FFFF99', '#FFFFCC', '#FFFFFF']

colors_rgb = list(map(ImageColor.getrgb, colors))
approximate_color = KDTree(colors_rgb)

def to_hex(rgb):
    rgb = tuple(int(c) for c in rgb)
    return '#%02x%02x%02x' % rgb

def nearest_color(rgbcolor):
    _, ind = approximate_color.query(rgbcolor)
    nearest_rgb = tuple(int(c) for c in approximate_color.data[ind])
    return to_hex(nearest_rgb)

def freqs_to_frame(freqs):
    tbl = [(c, freqs[c]) for c in freqs]
    tbl = pd.DataFrame(tbl, columns=('color', 'frequency'))
    return tbl.sort_values(by='frequency', ascending=False)[0:10]

def generate_color_histogram(image_path):
    img = Image.open(image_path).convert("RGBA")
    m, n = img.size
    color_freqs = {}
    npix = 0

    for i in range(m):
        for j in range(n):
            p = img.getpixel((i, j))
            if len(p) == 4 and p[3] == 0:
                continue
            p_rgb = p[0:3]
            p = nearest_color(p_rgb)
            if p in color_freqs:
                color_freqs[p] += 1
            else:
                color_freqs[p] = 1
            npix += 1

    for key in color_freqs:
        color_freqs[key] /= npix

    return freqs_to_frame(color_freqs)

def save_barplot(color_frame, filename):
    plt.figure(figsize=(10, 6))
    sns.barplot(data=color_frame, x='color', y='frequency', hue='color', dodge=False, palette=color_frame['color'].tolist(), legend=False)
    plt.xticks(rotation=45)
    plt.xlabel("")
    plt.ylabel("")
    plt.savefig(filename)
    plt.close()  # Make sure this is commented out if you want to display the plot in a Jupyter Notebook
    # plt.show()  # Uncomment this line for debugging to display the plot


def create_barplots_for_ads(directory):
    try:
        os.makedirs('barplots/')
    except FileExistsError:
        pass

    for filename in os.listdir(directory):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(directory, filename)
            color_frame = generate_color_histogram(image_path)
            plot_filename = os.path.join('barplots/', filename.split('.')[0] + '_barplot.png')
            save_barplot(color_frame, plot_filename)




In [28]:
# Google Colab drive mounting and directory settings remain unchanged
from google.colab import drive
drive.mount('/content/drive')
directory = "/content/drive/My Drive/Ads"  # Make sure this path is correct



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
create_barplots_for_ads(directory)